# Balance de Masa Superficial - Modelo de Lliboutry
### Autor: Rubén Basantes

### Instalación y carga de librerías

In [ ]:
source("Libraries.R")

### Carga de datos

In [ ]:
rm(list = ls())
rawData <- "D:/Proy_Revisiting/NL_Lliboutry/bmantisana15.txt"
shapes_path <- "D:/Proy_Revisiting/NL_Lliboutry/"
geo_path <- "D:/Proy_Revisiting/NL_Lliboutry/"
m <- 50 # range for the calculation of the balance by the weighted area method
Resolution <- 100 # It is the resolution of the raster to be used. Cell size = mxm units
N.Obs <- 6
crsID <- "EPSG:32717"
dem <- "D:/Proy_Revisiting/NL_Lliboutry/Antisana_ALOS.tif"
summit <- 5700

#### Crear un cuaderno de trabajo

In [ ]:
wb <- createWorkbook()

### Balance de masa según el modelo de Lliboutry

In [ ]:
source("funciones/Lliboutry_NL.R")
NLMR <- NLM(rawData, shapes_path, dem, Resolution, crsID, N.Obs, summit)
e_obs_mod <- NLMR$e_obs_mod
Nyear <- NLMR$Nyear
bit_LM <- NLMR$bit_LM
bit_LMf <- NLMR$bit_LMf
bo_t <- NLMR$bo_t
beta_it <- NLMR$beta_it
brick_raw <-NLMR$brick_raw
rawData <- NLMR$rawData
shap <- NLMR$shap
shape_list <- NLMR$shape_list
ai_df <- NLMR$ai_df
XY_coords <- NLMR$XY_coords
e_obs_mod_sd.tot <- sd(unlist(e_obs_mod[-c(1:3)]), na.rm=T)